In [1]:
import pandas as pd
import numpy as np

from models import CNN_Model

Using TensorFlow backend.


this is a version 46 features in total, no RVIS no mis_badness !


In [2]:
# combine files for training  

prefix = 'HIS' 

loc1 = '/home/local/ARCS/hq2130/missense/data/training/'
loc2 = '/home/local/ARCS/hq2130/missense/data/metaSVM/'
files = [loc1 + 'HGMD_DM_missense_norecceive.rare.'+prefix+'.reformat.csv',
         loc2 + 'metaSVM_train.anno.rare.'+prefix+'.reformat.csv',
         loc1 + 'CADD_neg_train.anno.rare.'+prefix+'.reformat.csv',
         loc1 + 'clinvar_pathogenic_1-4star.rare.'+prefix+'.reformat.csv',  
         loc1 + 'DiscovEHR_missense_sel.rare.'+prefix+'.reformat.csv']

df_list = []
for f_train in files:
    df = pd.read_csv(f_train)
    df_list.append(df)
df_combined = pd.concat(df_list, axis=0, ignore_index=True)

with open('../data/training/exclude_in_testing.txt') as f:
    exclude = set(line.strip() for line in f.readlines())
df_combined = df_combined[~df_combined.var_id.isin(exclude)]


# remove duplicates and overlapping variants 
df_combined = df_combined.drop_duplicates(subset='var_id', keep='first')
pos = df_combined[df_combined.target==1]
neg = df_combined[df_combined.target==0]
var_pos = set(pos.var_id)
var_neg = set(neg.var_id)
pos = pos[~pos.var_id.isin(var_neg)]
neg = neg[~neg.var_id.isin(var_pos)]
df = pd.concat([pos, neg], axis=0, ignore_index=True)
df.to_csv('../data/input_data.' + prefix + '.csv', index=False)

In [ ]:
weights_path = None
exclude_cols = {'var_id', 'aaref', 'aaalt', 'target', 'Ensembl_transcriptid',
                'ref', 'alt', 'category',
                'source', 'INFO', 'disease', 'genename',
                '#chr', 'pos(1-based)',  'hg19_chr', 'hg19_pos(1-based)',
                'CADD_phred', '1000Gp3_AF', 'ExAC_AF', 'gnomad',
                'RVIS', 'mis_badness', 'MPC', 'REVEL', 'domino'}

model = CNN_Model_Mode(weights_path=weights_path,
                       exclude_cols=exclude_cols,
                       train_flag=True,
                       verbose=2,
                       nb_epoch=40,
                       batch_size=64,
                       name='res_' + prefix,
                       fname='../data/input_data.' + prefix + '.csv',
                       f_out='../data/output/output_data_mode5.csv')


model.train(sub_sample=3)